In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from PIL import Image
from PIL import ImageFilter
import matplotlib as mpl
from PyPDF2 import PdfFileWriter, PdfFileReader
import io
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter,A4
%matplotlib inline

In [36]:
A4

(595.2755905511812, 841.8897637795277)

In [38]:
DATA = [
    {
        "zname": "Field 1",
        "lx": 300,
        "ly": 300,
        "rx": 320,
        "ry": 645,
        "page": 0,
        "page_height": 800,
        "page_width": 800
    },
    {
        "zname": "...Field 0",
        "lx": 0.5,
        "ly": 2.5,
        "rx": 50,
        "ry": 45,
        "page": 0,
        "page_height": 800,
        "page_width": 800
    } 
]

In [39]:
ZONES = []
for zone in DATA:
    while zone['page'] >= len(ZONES):
        ZONES.append([])
    ZONES[zone['page']].append(zone)    

In [45]:
output = PdfFileWriter()
old_pdf = PdfFileReader(open("graph.pdf","rb"))

for index,currPage in enumerate(ZONES):
    if len(currPage) == 0:
        continue
    
    packet = io.BytesIO()
    can = canvas.Canvas(packet)
    
    height = int(list(old_pdf.getPage(index).mediaBox)[3])
    width = int(list(old_pdf.getPage(index).mediaBox)[2])
    
    for zone in currPage:        
        hr = height/zone['page_height']
        wr = width/zone['page_width']
        
        if (width,height) == letter:
            #for us letters
            can.drawString(zone['lx']*wr-1,height - zone['ly']*hr,zone['zname'])  
        else:
            #for A4 size
            can.drawString(0.5*wr+zone['lx']*wr,height - zone['ly']*hr-10*hr,zone['zname'])

    can.save()

    packet.seek(0)
    new_pdf = PdfFileReader(packet)


    page = old_pdf.getPage(index)
    page.mergePage(new_pdf.getPage(0))
    output.addPage(page)

    outputStream = open("des.pdf","ab")
    output.write(outputStream)
    outputStream.close()    
    